### Modified from the [World Bank Example](https://worldbank.github.io/OpenNightLights/tutorials/mod4_1_time_series_charts.html)



# Time Series Charts

In the last module we worked through some basic operations and visualized the results as map layers. For analytical work, it is also useful to plot the data in charts. For understanding temporal dynamics, which is a common desire when working with remote sensing, you will want a line graph that shows quantities of a variable over time, commonly known as a time series graph.


**Our tasks in this exercise:**
1. Extracting VIIRS time series data and converting to a pandas dataframe
2. Create a 2014-2020 time series graph from VIIRS-DNB data for a point in Seoul, South Korea

## Extract time series data and convert to pandas dataframe

Those familiar with Python know that `pandas` is an indespensible library, a package data analysis built on another indispensible package, `numpy`. We will extract the data from our raster file of VIIRS-DNB radiance and convert it into a pandas dataframe in our local computing space so that we can use a plot libraries, `matplotlib` and `seaborn` to make our line graph.

First we define our point of interest: the location of Seoul Olympic Stadium (a.k.a. Jamsil Olympic Stadium). Technically we'll pick a lat/lon coordinate in the stadium and create a 500m buffer around it.

Then we'll grab a collection of VIIRS-DNB monthly composites from January 2014 to May 2020.

In [ ]:
# Authenticate and Initialize Earth Engine
import ee
# Authenticate as your Qwiklabs User
ee.Authenticate()
# Initialize the library by specifying the GCP project
your_project_id = ''
ee.Initialize(project=your_project_id)

In [ ]:
# Import required Libraries import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geemap

In [ ]:
# identify a 500 meter buffer around our Point Of Interest (POI)
poi = ee.Geometry.Point(127.072483, 37.515817).buffer(500)

viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2014-01-01','2020-5-31')

### Extracting all image values in the collection

To make a time series, we need to get all the values in our collection using the `map` function. We'll create a custom function in Python that takes a single image as an input and reduces the data in a given region (our point of interest in this case).

We'll get the mean of the pixels in our region and set the scale to 30. We'll use the `avg_rad` band.

We'll then need to set this reduced info as a property (we'll call it "mean") in our image so that the output of our function is to get the mean radiance of a particular region, and add this as a property on our image along with the date.

In [ ]:
def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=30).get('avg_rad')
    return img.set('date', img.date().format()).set('mean',mean)

We map this function to every image in our collection to get a new ImageCollection, but now each image has the mean value for the region of interest and the date. These are the data we'll make our time series out of.

In [ ]:
poi_reduced_imgs = viirs.map(poi_mean)

To convert to a pandas dataframe, we dont want an ImageCollection; however, so we will reduce our images to a list of lists:
- for each image, we have a 2-element list that contains that images date and mean value (for our point of interest)
- each of these lists are themselves elements in our outer list, which is what we'll convert to a dataframe

In [ ]:
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)

This nested list can be turned into a dataframe using the `.DataFrame` constructor. We'll name the columns, "date" and "mean".

In [ ]:
# dont forget we need to call the callback method "getInfo" to retrieve the data
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])

df

Great! Now we have a dataframe with each observation (month) as a row and it has the timestamp and mean values.

A reason pandas is so powerful is that it has built-ins, like a method to convert our string timestamp into a native timestamp data type.

We will also set our date column to be our index. Doing these things allows our `.plot` library to automatically convert the dates into nice readable dates on our axis.

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df = df.set_index('date')

df

OK! Now we're ready to make a plot.

## Create a time series graph for VIIRS-DNB from Jan 2014 to May 2020 for Seoul Olympic Stadium

We'll create a time series, but first let's get a look at where we're analyzing.

In [ ]:
koreaMap = geemap.Map()
koreaMap.centerObject(poi, zoom=15)
koreaMap.add_basemap("SATELLITE")
koreaMap.addLayer(poi, {}, "Seoul Olympic Stadium")
koreaMap.addLayerControl()
koreaMap

Our buffer encompasses almost the entire complex. Ok, let's look at times series graphs...

Pandas dataframes integrates nicely with our plotting libraries. We'll use `seaborn` to make a simple time series line plot.

In [ ]:
# we create a figure with pyplot and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15,7))

# we'll create the plot by setting our dataframe to the data argument
sns.lineplot(data=df, ax=ax)

# we'll set the labels and title
ax.set_ylabel('mean radiance',fontsize=20)
ax.set_xlabel('date',fontsize=20)
ax.set_title('Monthly mean radiance for Seoul Olympic Stadium (Jan 2014 to May 2020)',fontsize=20);

**Brief observations:** we see some strong seasonal patterns here (peaks and values), which is perhaps not too surprising for a sports stadium. We also see a spike in late 2018 as well as the drop-off in early 2020 that is very likely an indication of COVID-19 impacts.

Just out of curiousity...what is that spike? With dataframes we can easily find the date of that peak using the `.idxmax()` method to get the index (which is the month) of our maximum value.

In [ ]:
df.idxmax()

Hm, what was happening in September 2018? As it happens, 2018 is the 30th anniversary of the Seoul Olympics...perhaps there were a lot of events and concerts?

With our time series data, we can also smooth our time series (this is again, easy with Pandas. We'll get a moving avg of 12 months to smooth our data out and see overall trends):

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

# we'll plot the moving averate using ".rolling" and set a window of 12 months
window=12
sns.lineplot(data=df.rolling(window).mean(), ax=ax)

# we'll set the labels and title
ax.set_ylabel('mean radiance',fontsize=20)
ax.set_xlabel('date',fontsize=20)
ax.set_title(f'Monthly mean radiance ({window} moving avg.) for Seoul Olympic Stadium (Jan 2014 to May 2020)',fontsize=20);

We can see an overall trend that does seem to be decreasing, aside from our 2018 spike.

These's a lot you can do for time series analysis, including a Fourier Transform to identify strong cycles or even ARIMA models or more complex models to forecast trends. We wont get into that here, but hopefully you can see the power of remote sensing to support time series analysis.

At this point, the steps to map our reducer to our collection, reduce this further to nested lists, convert to a dataframe, and plot are the same, however!

Getting the sum (even at 500m scale) will probably take a few moments...even with the infrastructure of Google at work.